## Language Model (Rule-based & Probability-based)

#### 1. 基于规则的语言模型

第一个语法：

In [196]:
mission_grammar = '''
mission = 人名 ， 方式 去 地点 结尾词
人名 = Kitty | Jeremy | Ben | Alice
方式 = 骑马 | 坐车 | 弹射 | 游泳
地点 = 月球 | 火星 | 市中心 | 地心
结尾词 = 吗？ | 吧！
'''

第二个语法：

In [197]:
sentence_grammar = '''
sentence = NP VP
NP = Det Adj* Noun
Adj* = null | Adj Adj*
VP = V NP | V NP PP
PP = 在 NP Direction
Det = 这个 | 那个 | 这些 | 那些 | 一个 
Adj = 搞笑的 | 美丽的 | 胖胖的 | 湖南的 | 昏暗的
Noun = 女孩 | 男孩 | 公司 | 长凳 | 湖面 | 健身房
V = 看见 | 打 | 洗脸 | 起床 | 跑向
P = 在
Direction = 上 | 下 | 里 | 外
'''

In [198]:
# {key: list}
def create_grammar(grammar_str, split='=', line_split='\n'):
    grammar = {}
    for line in grammar_str.split('\n'):
        if not line.strip(): continue
        l = line.strip().split(split)
        grammar[l[0].strip()] = [form.split() for form in l[1].split('|')]
    return grammar

In [199]:
grammar1 = create_grammar(mission_grammar)
grammar1

{'mission': [['人名', '，', '方式', '去', '地点', '结尾词']],
 '人名': [['Kitty'], ['Jeremy'], ['Ben'], ['Alice']],
 '方式': [['骑马'], ['坐车'], ['弹射'], ['游泳']],
 '地点': [['月球'], ['火星'], ['市中心'], ['地心']],
 '结尾词': [['吗？'], ['吧！']]}

In [200]:
grammar2 = create_grammar(sentence_grammar)
grammar2

{'sentence': [['NP', 'VP']],
 'NP': [['Det', 'Adj*', 'Noun']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'VP': [['V', 'NP'], ['V', 'NP', 'PP']],
 'PP': [['在', 'NP', 'Direction']],
 'Det': [['这个'], ['那个'], ['这些'], ['那些'], ['一个']],
 'Adj': [['搞笑的'], ['美丽的'], ['胖胖的'], ['湖南的'], ['昏暗的']],
 'Noun': [['女孩'], ['男孩'], ['公司'], ['长凳'], ['湖面'], ['健身房']],
 'V': [['看见'], ['打'], ['洗脸'], ['起床'], ['跑向']],
 'P': [['在']],
 'Direction': [['上'], ['下'], ['里'], ['外']]}

In [201]:
import random

In [202]:
def generate(target, grammar):
    # base case
    if target not in grammar: return target
    expanded = [generate(p, grammar) for p in random.choice(grammar[target])]
    return ''.join([e for e in expanded if e != 'null'])

In [203]:
generate('mission', grammar1)

'Ben，骑马去地心吗？'

In [204]:
generate('mission', grammar1)

'Ben，弹射去地心吗？'

In [205]:
generate('sentence', grammar2)

'这些长凳洗脸那些湖面'

In [206]:
generate('sentence', grammar2)

'这些昏暗的湖南的湖南的湖面打这个昏暗的女孩'

In [207]:
def generate_n(n, grammar):
    sentences = []
    if grammar == grammar1:
        target = 'mission'
    elif grammar == grammar2:
        target = 'sentence'
    for i in range(n):
        sen = generate(target, grammar)
        sentences.append(sen)
    return sentences

In [208]:
generate_n(10, grammar1)

['Alice，弹射去地心吧！',
 'Kitty，骑马去市中心吗？',
 'Jeremy，骑马去市中心吧！',
 'Jeremy，游泳去月球吗？',
 'Alice，弹射去市中心吧！',
 'Ben，坐车去地心吧！',
 'Alice，坐车去火星吗？',
 'Alice，坐车去月球吧！',
 'Ben，骑马去月球吧！',
 'Ben，游泳去市中心吧！']

In [209]:
generate_n(20, grammar2)

['这些湖面打那些长凳',
 '这些湖面跑向这个昏暗的湖面',
 '这个健身房起床一个女孩',
 '那个男孩洗脸这些健身房在那个湖面里',
 '那个健身房看见那个湖面在那些美丽的美丽的湖南的湖面外',
 '一个男孩洗脸这些男孩在那些昏暗的搞笑的搞笑的公司上',
 '那些男孩起床这个美丽的昏暗的美丽的美丽的健身房',
 '那些健身房跑向那个女孩在一个昏暗的昏暗的公司上',
 '这些健身房看见这些男孩',
 '这个搞笑的公司洗脸这些湖南的湖面',
 '一个胖胖的公司洗脸这个湖面',
 '这些长凳洗脸一个搞笑的湖南的昏暗的昏暗的胖胖的男孩',
 '那些昏暗的长凳洗脸这些长凳在一个健身房外',
 '那个长凳洗脸那个胖胖的公司',
 '这些搞笑的男孩打这些长凳在这些健身房上',
 '这个搞笑的昏暗的搞笑的湖南的男孩洗脸这个长凳在那些公司外',
 '一个美丽的长凳洗脸这些长凳',
 '一个健身房看见这些长凳在这些湖面外',
 '一个美丽的湖面看见那个男孩',
 '那个搞笑的健身房看见这些公司在这个健身房上']

#### 2. 基于概率的语言模型

选用数据集：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv

In [210]:
import pandas as pd

In [211]:
filename = './movie_comments.csv'

In [212]:
content = pd.read_csv(filename, dtype={"id":"str", "link":"str", "name":"str", "comment":"str", "star":"str"})

In [213]:
content.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [214]:
comments = content['comment'].tolist()
comments[99]

'有一场坦克戏，简直令人浮想连连啊，吴京真的不是故意的嘛。又是压人压成肉泥的镜头，又是吴京站在坦克正前面，“稍有常识的人都会看出，如果敌方的铁骑继续前进”，然而吴京这个螳臂当车的歹徒真的阻挡住了。'

In [215]:
len(comments)

261497

In [216]:
import re

In [217]:
import jieba

In [218]:
def token(string):
    return re.findall('\w+', string)

In [219]:
from collections import Counter

In [220]:
with_jieba_cut = Counter(jieba.cut(comments[99]))

In [221]:
with_jieba_cut.most_common()[:10]

[('，', 6),
 ('的', 5),
 ('有', 2),
 ('坦克', 2),
 ('吴京', 2),
 ('真的', 2),
 ('。', 2),
 ('又', 2),
 ('是', 2),
 ('一场', 1)]

In [222]:
''.join(token(comments[99]))

'有一场坦克戏简直令人浮想连连啊吴京真的不是故意的嘛又是压人压成肉泥的镜头又是吴京站在坦克正前面稍有常识的人都会看出如果敌方的铁骑继续前进然而吴京这个螳臂当车的歹徒真的阻挡住了'

In [223]:
comments_clean = [''.join(token(str(c)))for c in comments]
comments_clean

['吴京意淫到了脑残的地步看了恶心想吐',
 '首映礼看的太恐怖了这个电影不讲道理的完全就是吴京在实现他这个小粉红的英雄梦各种装备轮番上场视物理逻辑于不顾不得不说有钱真好随意胡闹',
 '吴京的炒作水平不输冯小刚但小刚至少不会用主旋律来炒作吴京让人看了不舒服为了主旋律而主旋律为了煽情而煽情让人觉得他是个大做作大谎言家729更新片子整体不如湄公河行动1整体不够流畅编剧有毒台词尴尬2刻意做作的主旋律煽情显得如此不合时宜而又多余',
 '凭良心说好看到不像战狼1的续集完虐湄公河行动',
 '中二得很',
 '犯我中华者虽远必诛吴京比这句话还要意淫一百倍',
 '脑子是个好东西希望编剧们都能有',
 '三星半实打实的7分第一集在爱国主旋律内部做着各种置换与较劲但第二集才真正显露吴京的野心他终于抛弃李忠志了新增外来班底让硬件实力有机会和国际接轨开篇水下长镜头和诸如铁丝网拦截RPG弹头的细节设计都让国产动作片重新封顶在理念上它甚至做到绣春刀2最想做到的那部分',
 '开篇长镜头惊险大气引人入胜结合了水平不俗的快剪下实打实的真刀真枪让人不禁热血沸腾特别弹簧床架挡炸弹空手接碎玻璃弹匣割喉等帅得飞起就算前半段铺垫节奏散漫主角光环开太大等也不怕作为一个中国人两个小时弥漫着中国强大得不可侵犯的氛围还是让那颗民族自豪心砰砰砰跳个不停',
 '15100吴京的冷峰在这部里即像成龙又像杰森斯坦森但体制外的同类型电影主角总是代表个人无能的政府需要求助于这些英雄才能解决难题体现的是个人的价值所以主旋律照抄这种模式实际上是有问题的我们以前嘲笑个人英雄主义却没想到捆绑爱国主义的全能战士更加难以下咽',
 '犯我中华者虽远必诛是有多无脑才信这句话',
 '这部戏让人看的热血沸腾对吴京路转粉最后的彩蛋让我们没有理由不期待下一部',
 '假嗨特别恶心的电影',
 '有几处情节设置过于尴尬彰显国家自豪感的部分稍显突兀',
 '就是一部爽片打戏挺燃但是故事一般达康书记不合适这个角色赵东来倒是很合适张瀚太太太违和了分分钟穿越回偶像剧',
 '赵东来达康书记我们接到在非洲卧底的冷锋报告丁义珍现在非洲我们请求抓捕李达康东来这件事先不要声张特别是别让省厅知道就你和我一起去非洲加上冷锋同志三人逮捕丁义珍这次行就叫战狼2吧',
 '下一部拍喜剧吧整个片子真感觉挺搞笑的',
 '战狼2里吴京这么能打他打得过徐晓冬么',
 '

In [224]:
len(comments_clean)

261497

In [225]:
TOKEN = []

In [226]:
def cut(s): return list(jieba.cut(s))

In [227]:
for i, line in enumerate(comments_clean):
    if i % 100 == 0: print(i)
    
    # replace 10000 with a big number when you do your homework. 
    
    if i > 1000000: break    
    TOKEN += cut(line)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

134100
134200
134300
134400
134500
134600
134700
134800
134900
135000
135100
135200
135300
135400
135500
135600
135700
135800
135900
136000
136100
136200
136300
136400
136500
136600
136700
136800
136900
137000
137100
137200
137300
137400
137500
137600
137700
137800
137900
138000
138100
138200
138300
138400
138500
138600
138700
138800
138900
139000
139100
139200
139300
139400
139500
139600
139700
139800
139900
140000
140100
140200
140300
140400
140500
140600
140700
140800
140900
141000
141100
141200
141300
141400
141500
141600
141700
141800
141900
142000
142100
142200
142300
142400
142500
142600
142700
142800
142900
143000
143100
143200
143300
143400
143500
143600
143700
143800
143900
144000
144100
144200
144300
144400
144500
144600
144700
144800
144900
145000
145100
145200
145300
145400
145500
145600
145700
145800
145900
146000
146100
146200
146300
146400
146500
146600
146700
146800
146900
147000
147100
147200
147300
147400
147500
147600
147700
147800
147900
148000
148100
148200
148300

251600
251700
251800
251900
252000
252100
252200
252300
252400
252500
252600
252700
252800
252900
253000
253100
253200
253300
253400
253500
253600
253700
253800
253900
254000
254100
254200
254300
254400
254500
254600
254700
254800
254900
255000
255100
255200
255300
255400
255500
255600
255700
255800
255900
256000
256100
256200
256300
256400
256500
256600
256700
256800
256900
257000
257100
257200
257300
257400
257500
257600
257700
257800
257900
258000
258100
258200
258300
258400
258500
258600
258700
258800
258900
259000
259100
259200
259300
259400
259500
259600
259700
259800
259900
260000
260100
260200
260300
260400
260500
260600
260700
260800
260900
261000
261100
261200
261300
261400


In [228]:
words_count = Counter(TOKEN)
words_count.most_common(10)

[('的', 328262),
 ('了', 102420),
 ('是', 73106),
 ('我', 50338),
 ('都', 36255),
 ('很', 34712),
 ('看', 34022),
 ('电影', 33675),
 ('也', 32065),
 ('和', 31290)]

In [229]:
def prob_1(word):
    return words_count[word] / len(TOKEN)

In [230]:
prob_1('电影')

0.0074994772079362846

In [231]:
prob_1('吴京')

6.191105163493057e-05

In [232]:
prob_1('好看')

0.0018005426347784664

In [233]:
TOKEN = [str(t) for t in TOKEN]

In [234]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]

In [235]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [236]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2:
        return words_count_2[word1+word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

In [238]:
prob_2('这个','电影')

0.00011691840498353009

In [239]:
prob_2('很','好看')

0.00021446175999836092

In [240]:
prob_2('上升','很多')

2.2270172377815255e-07

In [241]:
# 2-gram language model
def get_probability(sentence):
    words = cut(sentence)
    sentence_pro = 1
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        prob = prob_2(word, next_)
        sentence_pro *= prob
    return sentence_pro

In [242]:
get_probability('这部电影的剧情非常有逻辑')

1.0670698233643085e-25

In [243]:
get_probability('剧情非常狗血')

5.157990008471098e-12

In [244]:
get_probability('国产动漫的水平已经提升了')

1.0063930397450096e-33

In [245]:
get_probability('非常好看的一部电影')

1.3014183941413956e-15

#### 3. 获得最优质的的语言

当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。

In [250]:
def generate_best(grammar, language_model, n): # you code here
    sentences = []
    for sentence in generate_n(n, grammar):
        sen_prob = language_model(sentence)
        sentences.append((sentence, sen_prob))
        print('sentence: {} with Prb: {}'.format(sentence, sen_prob))
    best = sorted(sentences, key=lambda x: x[1], reverse=True)[0]
    print('bset sentence: {} with Prb: {}'.format(best[0], best[1]))

In [251]:
generate_best(grammar1, get_probability, 20)

sentence: Alice，游泳去月球吧！ with Prb: 1.2199484279054584e-40
sentence: Kitty，弹射去地心吧！ with Prb: 1.2199484279054584e-40
sentence: Alice，骑马去火星吗？ with Prb: 1.2199484279054584e-40
sentence: Kitty，骑马去市中心吧！ with Prb: 1.2199484279054584e-40
sentence: Alice，骑马去火星吧！ with Prb: 3.659845283716375e-40
sentence: Alice，弹射去火星吧！ with Prb: 3.659845283716375e-40
sentence: Jeremy，弹射去月球吗？ with Prb: 1.2199484279054584e-40
sentence: Alice，弹射去火星吧！ with Prb: 3.659845283716375e-40
sentence: Ben，坐车去火星吧！ with Prb: 1.643384353576976e-33
sentence: Alice，弹射去市中心吧！ with Prb: 1.2199484279054584e-40
sentence: Jeremy，骑马去火星吧！ with Prb: 3.659845283716375e-40
sentence: Kitty，坐车去市中心吗？ with Prb: 5.477947845256587e-34
sentence: Jeremy，弹射去地心吧！ with Prb: 1.2199484279054584e-40
sentence: Jeremy，骑马去市中心吧！ with Prb: 1.2199484279054584e-40
sentence: Kitty，骑马去火星吗？ with Prb: 1.2199484279054584e-40
sentence: Jeremy，游泳去市中心吗？ with Prb: 1.2199484279054584e-40
sentence: Ben，骑马去市中心吗？ with Prb: 1.2199484279054584e-40
sentence: Jeremy，游泳去月球吧！ with 

In [252]:
generate_best(grammar2, get_probability, 20)

sentence: 这个胖胖的胖胖的男孩跑向这些搞笑的女孩 with Prb: 4.981180612565491e-55
sentence: 一个搞笑的湖南的昏暗的女孩跑向这个健身房 with Prb: 3.7946407133542526e-65
sentence: 那个女孩看见这些昏暗的女孩 with Prb: 8.471321883375503e-36
sentence: 那个公司看见那个长凳 with Prb: 2.459768946698195e-27
sentence: 这些胖胖的昏暗的搞笑的女孩洗脸这些湖南的长凳 with Prb: 1.4229902675078443e-65
sentence: 这个健身房看见那个湖面 with Prb: 2.459768946698195e-27
sentence: 那个湖面起床那些公司在那个公司下 with Prb: 9.499227335690975e-52
sentence: 那些长凳跑向那些公司 with Prb: 1.0955895690513175e-32
sentence: 这些男孩跑向一个美丽的昏暗的昏暗的胖胖的湖南的胖胖的搞笑的昏暗的公司 with Prb: 1.418205483682459e-113
sentence: 那些女孩看见那些搞笑的美丽的搞笑的搞笑的搞笑的美丽的美丽的公司在一个美丽的公司下 with Prb: 3.1411496714909434e-120
sentence: 那个昏暗的搞笑的长凳起床那些昏暗的湖南的女孩在那些搞笑的美丽的昏暗的搞笑的昏暗的湖面里 with Prb: 1.3017860898738818e-146
sentence: 这些昏暗的美丽的胖胖的胖胖的女孩洗脸一个胖胖的健身房 with Prb: 4.47887531280243e-68
sentence: 那些胖胖的搞笑的美丽的昏暗的男孩打这些湖南的胖胖的美丽的搞笑的女孩 with Prb: 5.257968429839558e-100
sentence: 一个昏暗的女孩打这些女孩在这些健身房外 with Prb: 4.911695133607098e-61
sentence: 那些健身房看见一个湖南的湖面 with Prb: 8.539638995338208e-40
sentence: 那个胖胖的昏暗

Q: 这个模型有什么问题？ 你准备如何提升？ 

Ans: We need a large corpus to calculate probability and ususally the probability will depend on the source of the corpus. Also, long sentences will yeild to small probability because each probability is less than one and we only calculate 2-gram.
We can use neural network to learn syntax and semantics to improve the model.